In [ ]:
import pandas as pd
from tenable.io import TenableIO
from pprint import pprint
import pandas as pd

In [ ]:
# Enable Tenable Log
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from tenable_authen import TenableAuthentication
credential = TenableAuthentication().getCredential()

TIO_ACCESS_KEY = credential["TENABLE_ACCESSKEY"]
TIO_SECRET_KEY = credential["TENABLE_SECRETKEY"]

tenableIO = TenableIO(TIO_ACCESS_KEY, TIO_SECRET_KEY)

del credential, TIO_ACCESS_KEY, TIO_SECRET_KEY

## Retrieve the list of scans

In [ ]:
def getlistOfScans():
    return tenableIO.scans.list()

In [ ]:
pd.DataFrame(getlistOfScans())

## Retrieve the Scans Detail

In [ ]:
def getScanDetail(scan_id=""):
    return tenableIO.scans.details(scan_id=scan_id)

In [ ]:
scan_id=44

In [ ]:
getScanDetail(scan_id=scan_id)

## Retrieve the list of scan histories

In [ ]:
def getlistOfScanhistories(scan_id=""):
    return tenableIO.scans.history(scan_id=scan_id)

In [ ]:
scan_id=44

In [ ]:
pd.DataFrame(getlistOfScanhistories(scan_id=scan_id))

## Retrieve the Scans Status Information

In [ ]:
def getScanStatusInformation(scan_id="", history_uuid=""):
    return tenableIO.scans.info(scan_id=scan_id, history_uuid=history_uuid )

In [ ]:
scan_id=44
history_uuid="3f1e276b-f5c3-4ffc-b870-39e11edabcd5"

In [ ]:
getScanStatusInformation(scan_id=scan_id, history_uuid=history_uuid)

## Create a New Scan

https://developer.tenable.com/docs/determine-settings-for-credential-type

In [ ]:
def createNewScan(name="", targets=[], credentials={}, template=None, compliance=None):
    return tenableIO.scans.create(name=name,
                                  targets=targets,
                                  credentials=credentials,
                                  template=template,
                                  compliance=compliance
                                 )

In [ ]:
name='Example Scan'
targets=['127.0.0.1']

credentials={'Host': {'SSH': [{'id': 'CREDENTIAL-UUID'}]}}
# credentials={'Host': {'Windows': [{'domain': '', 'username': 'Administrator', 'password': 'sekretsquirrel', 'auth_method': 'Password'}]}}

template = None
compliance = {}


In [ ]:
createNewScan(name=name, targets=targets, credentials=credentials, template=template, compliance=compliance)

## ManageStatus Scan by ScanId

In [ ]:
def startScan(scan_id=""):
    return tenableIO.scans.launch(scan_id=scan_id)

In [ ]:
def stopScan(scan_id=""):
    return tenableIO.scans.stop(scan_id=scan_id)

In [ ]:
def pauseScan(scan_id=""):
    return tenableIO.scans.pause(scan_id=scan_id)

In [ ]:
def resumeScan(scan_id=""):
    return tenableIO.scans.resume(scan_id=scan_id)

In [ ]:
def deleteScan(scan_id=""):
    return tenableIO.scans.delete(scan_id=scan_id)

## Get Status for latest Instance Scan

In [ ]:
def getStatusforlatestInstanceScan(scan_id=""):
    return tenableIO.scans.status(scan_id=scan_id)

In [ ]:
scan_id=44

In [ ]:
getStatusforlatestInstanceScan(scan_id=scan_id)

## Retrieve the scan results from either the latest scan or a specific scan instance in the history.

In [ ]:
def getScansResult(scan_id="", history_id=None, history_uuid=None):
    return tenableIO.scans.results(scan_id=scan_id, history_id=history_id, history_uuid=history_uuid)

In [ ]:
scan_id=44
history_id=None
history_uuid=None

In [ ]:
scanResult = getScansResult(scan_id=scan_id, history_id=history_id, history_uuid=history_uuid)
scanResult

In [ ]:
scanResult["info"]

In [ ]:
pd.DataFrame(scanResult["hosts"])

In [ ]:
pd.DataFrame(scanResult["vulnerabilities"])

In [ ]:
pd.DataFrame(scanResult["comphosts"])

In [ ]:
pd.DataFrame(scanResult["compliance"])

In [ ]:
pd.DataFrame(scanResult["filters"])

In [ ]:
scanResult["notes"]

In [ ]:
scanResult["remediations"]

## Export the scan report.

In [ ]:
def exportScansReport(scan_id="", history_id=None):
    with open('ScansReport.csv', 'wb') as reportobj:
        tenableIO.scans.export(scan_id=scan_id, history_id=history_id, format="csv", fobj=reportobj)

In [ ]:
scan_id=44
history_id=None

In [ ]:
exportScansReport(scan_id=scan_id, history_id=history_id)